Connect to correct port that the board is connected to
find the port like this on terminal:
```
ls /dev/tty.*
```

and send the entire lib folder to it with ampy like this on terminal (change to your path of the board):
```
ampy -p /dev/tty.usbserial-0001 put lib/
```

Make a file called .env with this inside it:

```
#.env
WIFI_SSID=WIFI_NAME
WIFI_PASSWORD=WIFI_PASSWORD
```
change WIFI_NAME and WIFI_PASSWORD to your WiFi credentials

and then run the following on terminal to send it to board (change to your path of the board):

```
ampy -p /dev/tty.usbserial-0001 put .env lib/.env
```

Now connect to the board on this notebook by running the next code, change path to your board port:

In [10]:
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


You can test if it is connecting to wifi here:

In [ ]:
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.env")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

Now you need to run the flask server by running on terminal, but change the path to your eeg_lib folder:
```
python /Users/rishil/Desktop/FYP/EEG-decoding/eeg_lib/simple_server.py
```

Inside the file you can change your output logs file name and path:
```
DEFAULT_FILENAME = "/Users/rishil/Desktop/FYP/EEG-decoding/eeg_lib/logs/rishil_test_1_raw.json"
```

You will need to send posts to the server with the code as followed, you can run it here to test:

In [3]:
import gc
from micropython import alloc_emergency_exception_buf
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)
# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

from ulab import numpy as np
import utime as time
from lib.runner import Runner  

runner = Runner('CCA', buffer_size=256)
runner.setup()
runner.run()

p26.off()
p13.off()

print(2)
time.sleep(5)

p26.on()
p13.on()

env_vars = load_env_vars("lib/.env")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

while True:
    time.sleep(4)
    data = runner.periph_manager.read_adc_buffer()
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/", toSend)
    del data

ADC initialised
SPI initialised
DigiPot set to 250 = gain of 2.031451
2
.network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')
{'raw_data': [1726, 1968, 2140, 1805, 1682, 1776, 2013, 2158, 2080, 1915, 1808, 2038, 2153, 2055, 1878, 1709, 1709, 2146, 2032, 1854, 1689, 1742, 1978, 2109, 1949, 1787, 2027, 2160, 2064, 1888, 1712, 1703, 2146, 2033, 1851, 1696, 1721, 1957, 2130, 1813, 1687, 1771, 2007, 2156, 2076, 1906, 1872, 2087, 2128, 2005, 1827, 1683, 1747, 2113, 1968, 1785, 1681, 1787, 2027, 2157, 1758, 1687, 1840, 2063, 2152, 2046, 1863, 1871, 2093, 2127, 1997, 1813, 1681, 1755, 2018, 1776, 1686, 1840, 2063, 2156, 2041, 1861, 1894, 2096, 2129, 2003, 1823, 1689, 1750, 2112, 1968, 1779, 1689, 1792, 2031, 2159, 1751, 1685, 1841, 2082, 2132, 1809, 1687, 1795, 2082, 1909, 1726, 1697, 1874, 2081, 2129, 1697, 1712, 2093, 2119, 1934, 1751, 1694, 2163, 2070, 1891, 1717, 1709, 1904, 2106, 1697, 1930, 2123, 2123, 1995, 1808, 1689, 1777, 2105, 1953, 1773, 1694, 1825, 205

Traceback (most recent call last):
  File "<stdin>", line 52, in <module>
  File "lib/requests.py", line 163, in JSONRequest
  File "lib/requests.py", line 381, in OpenRequestJSONData
  File "lib/requests.py", line 282, in OpenRequest
Exception: Error to connect to 192.168.0.13:5001


In [ ]:
runner.stop()

Now if it works on here then you need to make a file called main.py with that code in and send it to the board using:

Make sure source main.py is in same directory as this notebook

In [19]:
%sendtofile lib/runner.py --source lib/runner.py

Sent 252 lines (8517 bytes) to lib/runner.py.


In [14]:
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [15]:
def preprocess_data(signal):

    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.

    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    from lib.signal import sos_filter

    signal = np.array(signal) - np.mean(signal)  # remove DC component

    # downsample filtered signal by only selecting every `ds_factor` sample
    return sos_filter(signal, fs=self.base_sample_freq)

In [24]:
data = [1735, 1698, 1687, 1699, 1699, 1694, 1701, 1734, 1709, 1739, 1773, 1739, 1772, 1759, 1775, 1767, 1777, 1758, 1741, 1751, 1719, 1690, 1711, 1689, 1702, 1687, 1704, 1744, 1714, 1700, 1727, 1756, 1776, 1765, 1776, 1774, 1776, 1747, 1712, 1744, 1766, 1734, 1703, 1687, 1691, 1680, 1704, 1682, 1712, 1747, 1714, 1694, 1717, 1754, 1777, 1763, 1739, 1765, 1778, 1751, 1719, 1779, 1774, 1737, 1699, 1686, 1712, 1726, 1691, 1687, 1719, 1755, 1680, 1694, 1731, 1771, 1773, 1745, 1746, 1777, 1766, 1729, 1695, 1719, 1755, 1719, 1683, 1692, 1701, 1701, 1680, 1709, 1744, 1774, 1761, 1723, 1762, 1776, 1755, 1775, 1775, 1775, 1745, 1711, 1681, 1701, 1734, 1694, 1682, 1712, 1691, 1725, 1692, 1726, 1762, 1773, 1773, 1739, 1770, 1773, 1742, 1767, 1734, 1763, 1730, 1693, 1713, 1683, 1714, 1686, 1691, 1698, 1687, 1728, 1696, 1738, 1769, 1744, 1773, 1763, 1774, 1771, 1776, 1741, 1769, 1744, 1773, 1775, 1742, 1770, 1749, 1761, 1733, 1695, 1681, 1690, 1715, 1682, 1687, 1722, 1693, 1699, 1694, 1730, 1762, 1773, 1775, 1749, 1773, 1764, 1729, 1694, 1777, 1758, 1727, 1694, 1685, 1700, 1712, 1680, 1694, 1728, 1770, 1680, 1699, 1737, 1767, 1771, 1727, 1749, 1777, 1762, 1728, 1686, 1771, 1746, 1712, 1679, 1691, 1679, 1713, 1680, 1689, 1719, 1734, 1679, 1699, 1733, 1768, 1762, 1769, 1744, 1767, 1754, 1715, 1740, 1767, 1744, 1701, 1673, 1669, 1675, 1693, 1675, 1703, 1738, 1709, 1742, 1707, 1744, 1771, 1764, 1710, 1681, 1712, 1681, 1712, 1744, 1776, 1744, 1742, 1741, 1760, 1762, 1757, 1762, 1750, 1757, 1759, 1730, 1758, 1766, 1757, 1726, 1687, 1717]
# print(preprocess_data(self,data))

In [20]:
from ulab import numpy as np
import utime as time
from lib.runner import Runner  

runner = Runner('CCA', buffer_size=256)
runner.setup()
runner.run()


ADC initialised
SPI initialised
DigiPot set to 200 = gain of 4.895959610426813


In [26]:
print(len(runner.preprocess_data(data)))

64


In [23]:
runner.stop()